In [2]:
!pip install requests
!pip install beautifulsoup4

In [3]:
from bs4 import BeautifulSoup
import requests
import openpyxl
from openpyxl import Workbook
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows
from tqdm.auto import tqdm

In [18]:
wb = Workbook()
sheet = wb.active
sheet.append(['발행일','언론사','기사제목','URL','내용'])


headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102" }

#1, 11, 21.... => 1페이지는 1, 2페이지는 11, 3페이지는 21과 같이 10의 단위로 커짐
for page in tqdm(range(1501,1742,10)):
    # 1742
    # 501
    # 901
    # 1301
    # 1501

    #Get 요청, naver 서버에 대화 시도
    response = requests.get(f'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%EA%B5%90%EC%B4%8C%EC%B9%98%ED%82%A8&sort=1&photo=3&field=0&pd=3&ds=2015.01.01&de=2024.01.12&mynews=0&office_type=0&office_section_code=0&news_office_checked=&office_category=0&service_area=0&nso=so:dd,p:from20150101to20240112,a:all&start={page}')
    #네이버에서 html 제공, text 메소드로 태그 내 텍스트만 추출
    html = response.text

    #html 번역선생님으로 수프 만듦
    soup = BeautifulSoup(html, 'html.parser')
    news = soup.select('div.news_wrap.api_ani_send')
    for n in news:
        title = n.select_one('a.news_tit').text
        press = n.select_one('a.info.press').text
        #date = n.select_one('span.info').text

        try:
            url = n.select_one('div.info_group > a:nth-of-type(2)')['href']
        except:
            continue
        article = requests.get(url,headers = headers)
        article_html = BeautifulSoup(article.text,"html.parser")

        try:
            date = article_html.select_one('span.media_end_head_info_datestamp_time._ARTICLE_DATE_TIME').text
        except:
            continue
        try:
            content = article_html.select_one('#dic_area').text
        except:
            continue
        sheet.append([date, press, title,url, content])





#간격조절
sheet.column_dimensions['A'].width = 10
sheet.column_dimensions['B'].width = 10
sheet.column_dimensions['C'].width = 30
sheet.column_dimensions['D'].width = 40
sheet.column_dimensions['E'].width = 50
wb.save(filename='3월_22.xlsx')

  0%|          | 0/25 [00:00<?, ?it/s]